In [ ]:
import json
from mosaik_heatpump.heatpump.Heat_Pump_Des import Heat_Pump_Des
import pandas as pd
import numpy as np
from tqdm import tqdm
from tespy.tools import logger
import logging
logger.define_logging(log_path=True, log_version=True,
                      screen_level=logging.ERROR,
                      file_level=logging.DEBUG)

In [ ]:
# Enter the name of the new heat pump model. Should match the name used in
# the eta_s_data.json
model_name = 'Air_6kW'

In [ ]:
params_air = {
        'hp_model': model_name,
        'heat_source_T': 7,
        'cons_T': 35,
        'Q_Demand':10000,
        'calc_mode': 'detailed'
    }

heat_pump_1 = Heat_Pump_Des(params_air)

In [ ]:
with open("../../mosaik_heatpump/heatpump/eta_s_data.json") as feedsjson:
    data_1 = json.load(feedsjson)

etas_dict = data_1[heat_pump_1.hp_model]['eta_s']
heatload_dict = data_1[heat_pump_1.hp_model]['heatload']

T_min = min(list(map(int, etas_dict)))
T_max = max(list(map(int, etas_dict)))
# T_min = -7

hl_min = data_1[heat_pump_1.hp_model]['min_heatload']
hl_max = data_1[heat_pump_1.hp_model]['max_heatload']

cons_T_min = min(list(map(int, etas_dict[str(T_max)]))) -5
cons_T_max = max(list(map(int, etas_dict[str(T_max)]))) -5

T_amb_list = np.arange(T_min, T_max, 1)
cond_in_T_list = np.arange(cons_T_min, cons_T_max, 1)

In [ ]:
#Dict and DataFrame to save the fast mode data
final_dict = {}
df = pd.DataFrame(index=T_amb_list, columns=cond_in_T_list)

i = 1
total = len(T_amb_list)*len(cond_in_T_list)

pbar = tqdm(total=total)

for T_amb in T_amb_list:
    final_dict[T_amb] = {}
    for cond_in_T in cond_in_T_list:
        skip = False
        heat_pump_1.heat_source_T = T_amb
        heat_pump_1.cond_in_T = cond_in_T
        heat_pump_1._etas_heatload_id()
        try:
            heat_pump_1._design_hp()
        except:
            print('Error for T_amb : {} & Cond_in_T : {}'.format(T_amb, cond_in_T))
            skip = True
        heatload_list = np.arange(hl_min, heat_pump_1.heatload_des, 500)
        save_dict = {}

        for heatload in heatload_list:
            inputs = {'heat_source_T': T_amb, 'cond_in_T': cond_in_T, 'Q_Demand': heatload}

            if skip is False:
                try:
                    heat_pump_1.step(inputs)
                    save_dict[heatload] = {'cond_m': heat_pump_1.cond_m, 'COP': heat_pump_1.COP}
                    if heat_pump_1.cond_m == 0 and heat_pump_1.COP == 0:
                        print("Error for {} inputs".format(inputs))
                except:
                    print("Error for {} inputs".format(inputs))
                    save_dict[heatload] = {'cond_m': 0, 'COP': 0}
            else:
                skip = False
                print("Error for {} inputs".format(inputs))
                save_dict[heatload] = {'cond_m': 0, 'COP': 0}


        final_dict[T_amb][cond_in_T] = save_dict
        df.at[T_amb, cond_in_T] = save_dict
        pbar.update()

pbar.close()

In [ ]:
result = df.to_json(orient="index")
parsed = json.loads(result)
json.dumps(parsed, indent=4)

In [ ]:
with open("../../mosaik_heatpump/heatpump/cop_m_data.json") as feedsjson:
    feeds = json.load(feedsjson)

if model_name in feeds.keys():
    a = input('The model name already exists and must be replaced. Do you want to replace the data (y/n)? : ')
    if a.lower() == 'y':
        feeds[model_name]= parsed
    elif a.lower() =='n':
        model_name = input('Enter new model name')
        feeds[model_name]= parsed
    else:
        print("Wrong input. Execute cell again.")

In [ ]:
with open("../../mosaik_heatpump/heatpump/cop_m_data.json", "w") as write_file:
    json.dump(feeds, write_file, indent=4)